In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver.v2 as uc

driver = uc.Chrome(version_main=110)

In [7]:
company = '@ngic'

In [8]:
driver.get('https://licenseesearch.fldfs.com/')

In [9]:
radiobuttons = driver.find_elements(By.ID, 'EmailAddressBeginContainFilter')
for i, radio in enumerate(radiobuttons):
    if i == 1:
        radio.click()
        
emailfilter = driver.find_element(By.NAME, 'EmailFilter')
emailfilter.send_keys(company)

dropdown = Select(driver.find_element(By.ID, 'LicenseCategoryFilter'))
dropdown.select_by_visible_text('Adjuster')

search = driver.find_element(By.ID, 'btnSearch')
search.click()

WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

pagesraw = driver.find_element(By.ID, 'divMainSearchPaging')
pagewords = pagesraw.text.split()

numbers = []

for word in pagewords:
    if word.isdigit():
        numbers.append(int(word))

totalpages = numbers[1]  

In [10]:
data = []

for x in range(totalpages):
    pagesraw = driver.find_element(By.ID, 'divMainSearchPaging')
    pagewords = pagesraw.text.split()
    
    numbers = []

    for word in pagewords:
        if word.isdigit():
            numbers.append(int(word))

    currentpage = numbers[0]
    
    if currentpage - 1 != x:
        print('Wrong Page!')
        break
        
    table = driver.find_element(By.TAG_NAME, 'table')
    tbody = table.find_element(By.TAG_NAME, 'tbody')
    tablerows = tbody.find_elements(By.TAG_NAME, 'tr')
    
    for row in tablerows:
        link = row.find_element(By.TAG_NAME, 'a')
        link.click()
        
        valid_licenses_table = driver.find_element_by_xpath('//div[@class="form-group"]/div[@class="col-lg-6"][1]//table')
        
        if '(0720)' in valid_licenses_table.text and '(0620)' not in valid_licenses_table.text:
            adjuster = {}
            license = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[1]/div/div[1]/div').text
            name = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[1]/div/div[2]/div').text
            address = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[2]/div/div[1]/div').text 
            email = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[3]/div/div[1]/div').text
            phone = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[3]/div/div[2]/div').text
            county = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[4]/div/div[1]/div').text
            npn = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/div/div[4]/div/div[2]/div').text
            
            address = address.replace('\n', ' ')
            
            adjuster['Name'] = name
            adjuster['Email'] = email
            adjuster['Phone #'] = phone
            adjuster['Address'] = address
            adjuster['County'] = county
            adjuster['License #'] = license
            adjuster['NPN #'] = npn

            data.append(adjuster)
            driver.back()
        else:
            driver.back()
    
    if currentpage != totalpages:
        pageinput = driver.find_element(By.ID, 'inpPageGoToMain')
        pageinput.send_keys(currentpage + 1)

        submit = driver.find_element(By.XPATH, '//*[@id="divMainSearchPaging"]/span/input[2]')
        submit.click()
    
    driver.implicitly_wait(0.3)
    loading = driver.find_element(By.ID, 'loading-content')
    while loading.is_displayed():
        pass

<ipython-input-10-345afe863d8d>:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  valid_licenses_table = driver.find_element_by_xpath('//div[@class="form-group"]/div[@class="col-lg-6"][1]//table')


In [11]:
df = pd.DataFrame(data)
df

,Name,Email,Phone #,Address,County,License #,NPN #
0,"ABDELJAWAD, LARA",LARA.ABDELJAWAD@NGIC.COM,(336) 331-0832,"157 STONEBRIDGE BLVD APT 1621 EDMOND, OK 73013...",,W625645,16170443
1,"ADAMS, DESHAUNTA",SHAUN.ADAMS@NGIC.COM,(888) 233-4575,"500 WEST 5TH ST WINSTON SALEM, NC 27102",,W202185,8632691
2,"ADDIS, RACHEL",RACHEL.ADDIS@NGIC.COM,(216) 266-0711,"800 SUPERIOR CLEVELAND, OH 44114",,W366020,17575990
3,"ALFARO, JOSE DANIEL",JOSE.ALFARO@NGIC.COM,(336) 435-5562Ext. 55562,"5620 UNIVERSITY PKWY WINSTON SALEM, NC 27105",,W600947,19095063
4,"ALVAREZ-MARSH, SUSAN",SUSAN.MARSH@NGIC.COM,(216) 266-0614,"2528 CENTER RD HINCKLEY, OH 44233",,W410092,16611816
...,...,...,...,...,...,...,...
257,"WILLIAMS, TORA",TORA.WILLIAMS@NGIC.COM,(216) 912-4410,"6200 OAK TREE BLVD STE 200 INDEPENDENCE, OH 44...",,W322401,17867423
258,"WRIGHT, DEBORAH",DEBORAH.WRIGHT@NGIC.COM,(216) 350-5868,"800 SUPERIOR AVE CLEVELAND, OH 44114",,W423173,18481821
259,"YAWGER, JEREMY",JEREMY.YAWGER@NGIC.COM,(614) 854-2858,"5290 MOUNTAIN VIEW RD WINSTON SALEM, NC 271045116",,W439267,10037167
260,"ZEISLER, ERIC",ERIC.ZEISLER@NGIC.COM,(216) 357-7488,"800 SUPERIOR AVE E CLEVELAND, OH 441142613",,W176308,17107242


In [12]:
df.to_excel('Adjusters.xlsx', index=False)